# Example: Dip and Strike analysis using notebooks.

Thanks to Daven Quinn, https://davenquinn.com/


The example is based on: https://davenquinn.com/projects/attitude/example-notebooks/Plotting-Interactive.html

In [2]:
!pip install Attitude
!pip install palettable
!pip install pandas

You should consider upgrading via the 'c:\users\hs\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\hs\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\hs\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [1]:
import json
import requests
import numpy as np
import palettable as P
from attitude import Orientation, ReconstructedPlane, create_groups
from attitude.display import plot_interactive, init_notebook_mode


def queryPRo3D():
    annotationsAsString = requests.get('http://localhost:4321/api/integration/geojson_latlon').content
    annotations = json.loads(annotationsAsString)

    orientations = []
    for annotation in annotations["features"]:
        coords = annotation["geometry"]["coordinates"]
        if len(coords) > 2:
            orientation = Orientation(np.array(coords)) 

            orientations.append(orientation) 

    return orientations

    

## Get Data from PRo3D/PROVEX

In [2]:
orientations = queryPRo3D()

### Process data (depending on use case)

In [3]:
# the original example groups some orientations, so we group also some to see how this works
hashes = [ o.hash for o in orientations ] # fancy way for iterating a list and returning values into a new list
groups = (
    [hashes[0],hashes[1]], # group first and second orientation
    [hashes[2],hashes[2]] # group third orientation with itself (there are only three orientations in our data)
)

groupedOrientations = create_groups(orientations, *groups, same_plane=False) # create the groups
collection = [ go.to_mapping(color='#ff0000', type='remote') for go in groupedOrientations] # to_mapping converts the orientations to a suitable format for the final plotting, but I don't know what this type is for

import inspect
cmap = P.mycarta.CubeYF_7

heights = np.array([o["center"][2] for o in collection])
rng = [heights.min(),heights.max()]

for o in collection:
    ix = np.interp(o['center'][2], rng, [0,6])
    o['color'] = cmap.hex_colors[6-int(ix)]

## Visualize using attitude

In [5]:
# plot everything in an interactive plot
init_notebook_mode()
plot_interactive(collection)